In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Potholes/yolov7')
os.getcwd()

'/content/drive/MyDrive/Potholes/yolov7'

In [ ]:
# !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt

In [ ]:
!python train.py --epochs 80 --workers 8 --device 0 --batch-size 25 --data data.yaml --img 640 640 --cfg cfg/training/yolov7.yaml --weights 'yolov7-tiny.pt' --name YOLO_pothole --hyp data/hyp.scratch.p5.yaml


2024-05-17 11:58:53.069155: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 11:58:53.069208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 11:58:53.070677: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-17 11:58:54.421516: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
YOLOR 🚀 v0.1-128-ga207844 torch 2.2.1+cu121 CUDA:0 (Tesla T4, 15102.0625MB)

Namespace(weights='yolov7-tiny.pt', cfg='cfg/training/yolov7.yaml', data='data.yaml', hyp='data/hyp.scratch.p5.yaml', epochs=80, batch_size=25, img_size=[640, 640],

In [ ]:
import torch
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
model = torch.hub.load('/content/drive/MyDrive/Potholes/yolov7', 'custom', path_or_model ='/content/drive/MyDrive/Potholes/yolov7/runs/train/YOLO_pothole25/weights/best.pt', source='local')


requirements: numpy<1.24.0,>=1.18.5 not found and is required by YOLOR, attempting auto-update...


KeyboardInterrupt: 

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    # Make detections
    results = model(frame)

    cv2.imshow('YOLO', np.squeeze(results.render()))

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
!python detect.py --weights /content/drive/MyDrive/Potholes/yolov7/runs/train/YOLO_pothole25/weights/best.pt --img 640 --source /content/drive/MyDrive/Potholes/yolov7/pothol_test2.mp4 --save-txt --conf 0.3


In [ ]:
import cv2

# Load pre-trained YOLO model
net = cv2.dnn.readNet("/content/drive/MyDrive/Potholes/yolov7/runs/train/YOLO_pothole23/weights/best.pt", "/content/drive/MyDrive/Potholes/yolov7/cfg")
classes = ["pothole"]  # Class names used for training (you may have more classes)

# Capture live video stream
cap = cv2.VideoCapture(0)  # Use 0 for webcam, or path to video file

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess frame
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Perform inference
    detections = net.forward()

    # Loop over detections
    for detection in detections:
        # Extract class ID and confidence
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        # Filter detections by confidence threshold
        if confidence > 0.5 and class_id == 0:  # Assuming 0 is the class ID for pothole
            # Extract bounding box coordinates
            box = detection[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (x, y, w, h) = box.astype("int")

            # Draw bounding box and label on frame
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            text = f"Pothole: {confidence:.2f}"
            cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display frame
    cv2.imshow("Live Stream", frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [ ]:
from geopy.geocoders import Nominatim

def get_location():
    try:
        geolocator = Nominatim(user_agent="my_geocoder")
        ip_address = input("Enter IP Address (Leave blank for current device IP): ")
        if not ip_address:
            # If no IP address is provided, use the IP address of the device running the script
            import socket
            ip_address = socket.gethostbyname(socket.gethostname())
        location = geolocator.geocode(ip_address)
        if location:
            print("Location found for IP address:", ip_address)
            print("Latitude:", location.latitude)
            print("Longitude:", location.longitude)
        else:
            print("Location not found for IP address:", ip_address)
    except Exception as e:
        print("An error occurred:", str(e))

if __name__ == "__main__":
    get_location()
